In [1]:
import pandas as pd
import numpy as np
from helpers import calibrate_train_clfs, load_data, get_baseline, print_results, run_clf

In [2]:
keep_columns =  ['IsTraining','FTR', 'Div', 'Date', 'HomeTeam', 'AwayTeam', 
    'away_away_team_corners_against', 'away_away_team_corners_for',	'away_away_team_goals_against',	'away_away_team_goals_for',	
    'away_away_team_possession',	'away_away_team_shotsoff_against',	'away_away_team_shotsoff_for',	'away_away_team_shotson_against',
    	'away_away_team_shotson_for',	'away_direct_team_corners_against',	'away_direct_team_corners_for',	'away_direct_team_goals_against',
        'away_direct_team_goals_for',	'away_direct_team_possession',	'away_direct_team_shotsoff_against',	'away_direct_team_shotsoff_for',
        'away_direct_team_shotson_against',	'away_direct_team_shotson_for',	'home_direct_team_corners_against',	'home_direct_team_corners_for',	
        'home_direct_team_goals_against',	'home_direct_team_goals_for',	'home_direct_team_possession',	'home_direct_team_shotsoff_against',
        'home_direct_team_shotsoff_for',	'home_direct_team_shotson_against',	'home_direct_team_shotson_for',	
        'home_home_team_corners_against',	'home_home_team_corners_for',	'home_home_team_goals_against',	'home_home_team_goals_for',	
        'home_home_team_possession',	'home_home_team_shotsoff_against',	'home_home_team_shotsoff_for',	'home_home_team_shotson_against',
                 'home_home_team_shotson_for','Referee' 
]
 
drop_columns = [
    'B365H', 'B365D', 'B365A',
    'AC', 'AF', 'AR', 'AS', 'AST', 'AY', 
    'BWA', 'BWD', 'BWH', 'Bb1X2', 'BbAH', 'BbAHh', 'BbAv<2.5', 'BbAv>2.5', 'BbAvA', 'BbAvAHA', 
    'BbAvAHH', 'BbAvD', 'BbAvH', 'BbMx<2.5', 'BbMx>2.5', 'BbMxA', 'BbMxAHA', 'BbMxAHH', 'BbMxD',
    'BbMxH', 'BbOU', 'FTAG', 
    'HC', 'HF', 'HR', 'HS', 'HST', 
    'HTAG', 'HTHG', 'HTR', 'HY', 'IWA', 'IWD', 'IWH', 
    'LBA', 'LBD', 'LBH', 'PSA', 'PSCA', 'PSCD', 'PSCH', 'PSD', 'PSH', 
    'VCA', 'VCD', 'VCH', 'WHA', 'WHD', 'WHH', 'GoalFirstHalf', 'SHHG', 'SHAG', 'GoalSecondHalf','FTR','BothToScore',
]

#
target_column = "FTR"
drop_columns.remove(target_column)
X,y = load_data(columns_to_drop=drop_columns, is_training=True, target_name=target_column)

KEEEP
LENGTH1180


In [3]:
type(y)

pandas.core.series.Series

In [4]:
import numpy
import pandas
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.utils import np_utils
from keras.layers import Bidirectional, LSTM
from keras.initializers import glorot_normal

from sklearn.model_selection import cross_val_score, train_test_split, TimeSeriesSplit
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

C:\Users\martin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
y=y.values
X=X.values

encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [6]:
from keras import backend as K
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(12,  activation='relu')) # input_dim=314,
#   model.add(Bidirectional(LSTM(256,  return_sequences=True, dropout=0.1, recurrent_dropout=0.1,kernel_initializer=glorot_normal(seed=None)),name = 'BDLSTM1')) #Best = 300,0.25,0.25
#    model.add(Dense(8, activation='relu'))
#    model.add(Dense(1, activation='linear'))
    model.add(Dense(3, activation='softmax'))
#     # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'mae', 'mse', f1_m], verbose=1)
    #model.compile(loss='mse', optimizer='adam', metrics=['mse','mae']) 
    return model
estimator = KerasClassifier(build_fn=baseline_model, epochs=10, batch_size=2, verbose=1)

In [7]:
# #seed = 7
# #numpy.random.seed(seed)
# from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)# max_train_size=100
# for train_index, test_index in tscv.split(X):
#     print("TRAIN:", train_index.shape, "TEST:", test_index.shape)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

# #kfold = KFold(n_splits=10)
# # shuffle=True, random_state=seed
results = cross_val_score(estimator, X, dummy_y, cv=tscv.split(X))

Epoch 1/10
295/295 [==============================] - 41s 138ms/step - loss: 3.8092 - acc: 0.3186 - mean_absolute_error: 0.4573 - mean_squared_error: 0.3513 - f1_m: 0.2294
Epoch 2/10
295/295 [==============================] - 1s 5ms/step - loss: 1.1269 - acc: 0.3932 - mean_absolute_error: 0.4381 - mean_squared_error: 0.2231 - f1_m: 0.0475
Epoch 3/10
295/295 [==============================] - 1s 4ms/step - loss: 1.0803 - acc: 0.4000 - mean_absolute_error: 0.4373 - mean_squared_error: 0.2185 - f1_m: 0.0316
Epoch 4/10
295/295 [==============================] - 1s 3ms/step - loss: 1.0735 - acc: 0.4136 - mean_absolute_error: 0.4333 - mean_squared_error: 0.2169 - f1_m: 0.0395
Epoch 5/10
295/295 [==============================] - 2s 7ms/step - loss: 1.0771 - acc: 0.4068 - mean_absolute_error: 0.4363 - mean_squared_error: 0.2177 - f1_m: 0.0271
Epoch 6/10
295/295 [==============================] - 2s 6ms/step - loss: 1.0671 - acc: 0.4034 - mean_absolute_error: 0.4322 - mean_squared_error: 0.215

In [8]:
results

array([0.46779661, 0.4779661 , 0.61016949])

In [9]:
import statistics 
x = statistics.mean(results)
x

0.5186440677966102

In [10]:
X_train_calibrate, X_test, y_train_calibrate, y_test = train_test_split(X, y, test_size=0.20)
X_train, X_calibrate, y_train, y_calibrate = train_test_split(X_train_calibrate, y_train_calibrate, test_size = 0.3, random_state = 42)

In [11]:
model = baseline_model()
model.fit(X, dummy_y, nb_epoch=30, batch_size=10, validation_split=0.2, verbose=1)

C:\Users\martin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 944 samples, validate on 236 samples
Epoch 1/30
944/944 [==============================] - 1s 891us/step - loss: 4.1613 - acc: 0.3464 - mean_absolute_error: 0.4313 - mean_squared_error: 0.3411 - f1_m: 0.3218 - val_loss: 2.4166 - val_acc: 0.2881 - val_mean_absolute_error: 0.4686 - val_mean_squared_error: 0.3456 - val_f1_m: 0.2718
Epoch 2/30
944/944 [==============================] - 1s 565us/step - loss: 1.2422 - acc: 0.4206 - mean_absolute_error: 0.4248 - mean_squared_error: 0.2371 - f1_m: 0.2397 - val_loss: 2.6266 - val_acc: 0.2754 - val_mean_absolute_error: 0.4813 - val_mean_squared_error: 0.3812 - val_f1_m: 0.2642
Epoch 3/30
944/944 [==============================] - 0s 484us/step - loss: 1.1713 - acc: 0.4407 - mean_absolute_error: 0.4215 - mean_squared_error: 0.2292 - f1_m: 0.2323 - val_loss: 1.4260 - val_acc: 0.3347 - val_mean_absolute_error: 0.4383 - val_mean_squared_error: 0.2714 - val_f1_m: 0.2548
Epoch 4/30
944/944 [==============================] - 1s 665us/step - lo

Epoch 29/30
944/944 [==============================] - 0s 497us/step - loss: 0.8398 - acc: 0.5805 - mean_absolute_error: 0.3308 - mean_squared_error: 0.1661 - f1_m: 0.5024 - val_loss: 1.0377 - val_acc: 0.6144 - val_mean_absolute_error: 0.2933 - val_mean_squared_error: 0.1751 - val_f1_m: 0.6315
Epoch 30/30
944/944 [==============================] - 1s 613us/step - loss: 0.8138 - acc: 0.6049 - mean_absolute_error: 0.3219 - mean_squared_error: 0.1603 - f1_m: 0.5400 - val_loss: 1.2083 - val_acc: 0.5424 - val_mean_absolute_error: 0.3043 - val_mean_squared_error: 0.2154 - val_f1_m: 0.5434


In [12]:
prediction = model.predict(X[0:100])
prediction /= prediction.sum(axis=1).reshape((-1, 1))
prediction

array([[9.8805410e-01, 1.1120364e-02, 8.2555437e-04],
       [3.1520617e-01, 2.8319022e-01, 4.0160367e-01],
       [3.1520617e-01, 2.8319022e-01, 4.0160367e-01],
       [3.1520617e-01, 2.8319022e-01, 4.0160367e-01],
       [3.1520617e-01, 2.8319022e-01, 4.0160367e-01],
       [9.8385274e-01, 1.5235739e-02, 9.1151951e-04],
       [3.1520617e-01, 2.8319022e-01, 4.0160367e-01],
       [3.1520617e-01, 2.8319022e-01, 4.0160367e-01],
       [3.1520617e-01, 2.8319022e-01, 4.0160367e-01],
       [3.1520617e-01, 2.8319022e-01, 4.0160367e-01],
       [3.1520617e-01, 2.8319022e-01, 4.0160367e-01],
       [3.1520617e-01, 2.8319022e-01, 4.0160367e-01],
       [9.8017597e-01, 5.9934468e-03, 1.3830588e-02],
       [3.1520617e-01, 2.8319022e-01, 4.0160367e-01],
       [3.1520617e-01, 2.8319022e-01, 4.0160367e-01],
       [3.1520617e-01, 2.8319022e-01, 4.0160367e-01],
       [3.1520617e-01, 2.8319022e-01, 4.0160367e-01],
       [3.1520617e-01, 2.8319022e-01, 4.0160367e-01],
       [3.1520617e-01, 2.831